In [1]:
import utils.bit_tools as bit
import utils.qonvert_tools as qonvert
import utils.linalg_tools as la
import utils.circuit_tools as crc
import utils.cs_vqe_tools as c_tools

import cs_vqe_classes.cs_vqe as c
import cs_vqe_classes.eigenstate as eig


import ast
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from copy import deepcopy
from openfermion.linalg import LinearQubitOperator, get_sparse_operator, get_ground_state
import itertools
from statistics import median

from qiskit.circuit.parameter import Parameter
from qiskit.circuit.quantumcircuit import QuantumCircuit
from qiskit import QuantumCircuit, BasicAer, execute
from qiskit.visualization import plot_histogram
from qiskit.utils import QuantumInstance
from qiskit.aqua.components.optimizers import SLSQP
from qiskit.algorithms import VQE
from qiskit import Aer

In [2]:
import openfermion
import openfermionpyscf
from openfermion import MolecularData
from openfermionpyscf import run_pyscf
from openfermion.ops import FermionOperator, QubitOperator
from openfermion.transforms import jordan_wigner, bravyi_kitaev
from openfermion.transforms import get_fermion_operator
from openfermion.circuits import (uccsd_singlet_get_packed_amplitudes,
                               uccsd_singlet_generator, uccsd_generator,
                               uccsd_convert_amplitude_format)

singlet_bool = True # Set general UCCSD or singlet UCCSD.

bond_len = 0.772#1.45
atom_1 = 'He'
atom_2 = 'H'
basis = '3-21g'
multiplicity = 1
charge = 1

coordinate_1 = (0.0, 0.0, 0.0)
coordinate_2 = (0.0, 0.0, bond_len)
geometry = [(atom_1, coordinate_1), (atom_2, coordinate_2)]

molecule_data = MolecularData(geometry, basis, multiplicity, charge, description='Test')
#molecule.load()

# Set calculation parameters.
run_scf = 1
run_mp2 = 1
run_cisd = 0
run_ccsd = 0
run_fci = 1
delete_input = True
delete_output = True

# Run pyscf.
molecule = run_pyscf(molecule_data,
                     run_scf=run_scf,
                     run_mp2=run_mp2,
                     run_cisd=run_cisd,
                     run_ccsd=run_ccsd,
                     run_fci=run_fci)

#molecule.load()
#print(molecule)

ham = get_fermion_operator(molecule.get_molecular_hamiltonian())
ham_q = jordan_wigner(ham)
print('Hamiltonian:', '\n', ham_q, '\n')

scf = True      # Hartree-Fock.
mp2 = True      # Moller-Plesset 2.
cisd = True     # Configuration interaction singles and doubles.
ccsd = True     # Coupled cluster singles and doubles.
fci = True      # Full configuration interaction.

calculated_molecule = run_pyscf(molecule_data, scf, mp2, cisd, ccsd, fci)

if ccsd:
    ccsd_single_amps = calculated_molecule.ccsd_single_amps
    ccsd_double_amps = calculated_molecule.ccsd_double_amps

num_electrons = calculated_molecule.n_electrons
num_qubits = 2*calculated_molecule.n_orbitals

if singlet_bool:
    # Get singlet UCCSD generator.
    packed_amps = uccsd_singlet_get_packed_amplitudes(ccsd_single_amps,  ccsd_double_amps, num_qubits, num_electrons)
    ucc_sing = uccsd_singlet_generator(packed_amps, num_qubits, num_electrons)
    #print(ucc_sing)

else:
    # Get general UCCSD operator.
    ucc_op = uccsd_generator(ccsd_single_amps, ccsd_double_amps)
    #print(ucc_op)
    
ucc_q = jordan_wigner(ucc_sing)
ucc = qonvert.QubitOperator_to_dict(ucc_q, num_qubits)
print('UCCSD ansatz:', '\n', ucc_q)

Hamiltonian: 
 (1.132443402106938+0j) [] +
(-0.02581025441479668+0j) [X0 X1 Y2 Y3] +
(-0.012799934786443819+0j) [X0 X1 Y2 Z3 Z4 Y5] +
(-0.025159645839889756+0j) [X0 X1 Y2 Z3 Z4 Z5 Z6 Y7] +
(-0.012799934786443819+0j) [X0 X1 X3 X4] +
(-0.025159645839889756+0j) [X0 X1 X3 Z4 Z5 X6] +
(-0.01711452882177827+0j) [X0 X1 Y4 Y5] +
(-0.000989104910375095+0j) [X0 X1 Y4 Z5 Z6 Y7] +
(-0.000989104910375095+0j) [X0 X1 X5 X6] +
(-0.05469856574685882+0j) [X0 X1 Y6 Y7] +
(0.02581025441479668+0j) [X0 Y1 Y2 X3] +
(0.012799934786443819+0j) [X0 Y1 Y2 Z3 Z4 X5] +
(0.025159645839889756+0j) [X0 Y1 Y2 Z3 Z4 Z5 Z6 X7] +
(-0.012799934786443819+0j) [X0 Y1 Y3 X4] +
(-0.025159645839889756+0j) [X0 Y1 Y3 Z4 Z5 X6] +
(0.01711452882177827+0j) [X0 Y1 Y4 X5] +
(0.000989104910375095+0j) [X0 Y1 Y4 Z5 Z6 X7] +
(-0.000989104910375095+0j) [X0 Y1 Y5 X6] +
(0.05469856574685882+0j) [X0 Y1 Y6 X7] +
(0.002577905179360209+0j) [X0 Z1 X2] +
(0.00476614810567948+0j) [X0 Z1 X2 X3 Z4 X5] +
(-0.009264702923296737+0j) [X0 Z1 X2 X3 Z4 Z5 Z6 

UCCSD ansatz: 
 0.006478936130102838j [X0 X1 X2 Y3] +
0.006478936130102838j [X0 X1 Y2 X3] +
0.0020562744029710704j [X0 X1 X4 Y5] +
0.0020562744029710704j [X0 X1 Y4 X5] +
0.005083283191416048j [X0 X1 X6 Y7] +
0.005083283191416048j [X0 X1 Y6 X7] +
-0.006478936130102838j [X0 Y1 X2 X3] +
0.006478936130102838j [X0 Y1 Y2 Y3] +
-0.0020562744029710704j [X0 Y1 X4 X5] +
0.0020562744029710704j [X0 Y1 Y4 Y5] +
-0.005083283191416048j [X0 Y1 X6 X7] +
0.005083283191416048j [X0 Y1 Y6 Y7] +
-0.006360390047729467j [X0 Z1 Y2] +
-0.002645411873769119j [X0 Z1 Z2 Z3 Y4] +
0.0009221551709274561j [X0 Z1 Z2 Z3 Z4 Z5 Y6] +
-0.006478936130102838j [Y0 X1 X2 X3] +
0.006478936130102838j [Y0 X1 Y2 Y3] +
-0.0020562744029710704j [Y0 X1 X4 X5] +
0.0020562744029710704j [Y0 X1 Y4 Y5] +
-0.005083283191416048j [Y0 X1 X6 X7] +
0.005083283191416048j [Y0 X1 Y6 Y7] +
-0.006478936130102838j [Y0 Y1 X2 Y3] +
-0.006478936130102838j [Y0 Y1 Y2 X3] +
-0.0020562744029710704j [Y0 Y1 X4 Y5] +
-0.0020562744029710704j [Y0 Y1 Y4 X5] +
-0.0

In [3]:
ham = qonvert.QubitOperator_to_dict(ham_q, num_qubits)
terms_noncon = c_tools.greedy_dfs(ham, 1, criterion='weight')[-1]

In [4]:
mol = c.cs_vqe(ham, terms_noncon, num_qubits)
ham_noncon = mol.get_ham(h_type='noncon', rot=True)
gs_noncon_energy = mol.gs_noncon_energy()

In [5]:
A = mol.generators(rot=True)[1]
G = mol.generators(rot=True)[0]
print(A)
r1 = list(A.values())[0]
r2 = list(A.values())[1]

{'IZZZZZXI': 0.04017211880921176, 'IIIIIIZI': 0.9991927746287893}


In [14]:
initial_state='00010000'
index = bit.bin_to_int(initial_state)

t1 = eig.eigenstate(A, index, num_qubits).t_val()
t2 = - np.arctan((1+r2)/r1)

print(t1, t2)

-1.5507048609856555 -1.550704860985656


In [12]:
import legacy.eigenstate_generator as eig_legacy

psi_1 = eig_legacy.add_eigenstate('HeH+', r1, r2, 16, num_qubits, theta=0, custom_amp=None, rot=True)
for index, e in enumerate(psi_1):
    if e != 0:
        print(index, e)

16 (-0.9997981732901869+0j)
18 (0.020090114126239778+0j)


In [13]:
print(eig.eigenstate(A, 16, num_qubits).P_index())
psi_2 = eig.eigenstate(A, 16, num_qubits).construct()
for index, e in enumerate(psi_2):
    if e != 0:
        print(index, e)

{'X1': [6], 'X2': [], 'Y1': [], 'Y2': [], 'Z1': [1, 2, 3, 4, 5], 'Z2': [6]}
16 -0.9997981732901869
18 0.020090114126240222


In [9]:
G

{'ZIIIIIII': 1.0,
 'IZIIIIII': 1.0,
 'IIZIIIII': 1.0,
 'IIIZIIII': -1.0,
 'IIIIZIII': 1.0,
 'IIIIIZII': 1.0,
 'IIIIIIIZ': 1.0}

In [15]:
ham_q = qonvert.dict_to_QubitOperator(mol.get_ham(rot=True))
ham_noncon_q = qonvert.dict_to_QubitOperator(ham_noncon)
initial_state='00010000'
index = bit.bin_to_int(initial_state)
print(index)

#take expectation value algebraically
psi = eig.eigenstate(A, index, num_qubits).construct()
#psi = eig.rotate_state(rotations, psi, num_qubits)
expect = la.expectation(ham_q, psi, num_qubits)

print('<H> w.r.t. simultaneous eigenstate:', expect)
print('Noncontextual approximation:', gs_noncon_energy)
print('Expectation values match?', abs(expect - gs_noncon_energy) < 10**-12) #sanity check - expectation values match?

16
<H> w.r.t. simultaneous eigenstate: (-3.14282474926816+0j)
Noncontextual approximation: -3.142824749268162
Expectation values match? True


In [11]:
psi = eig.eigenstate(A, 52, num_qubits).construct()

In [12]:
A_q = qonvert.dict_to_QubitOperator(A)

In [14]:
la.expectation(A_q, psi, num_qubits)

(1+0j)